<h1 style="background-color:black;font-family:segoe ui;color:#FFF9ED;font-size:300%;text-align:center;border-radius:10px 10px;"> 
🌐 הסבת גושים (מוסדרים/לא מוסדרים) לבנק"ל מודרני 🌐

In [ ]:
print('1. Blocks Borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  1.a Reading inputs')

In [ ]:
RawData_folder = r'\\mapi_shares\MNCDB\צוות מידע\RawData/'

<h1 style="text-align:right;font-size:100%">
טבלת גושים מאורקל

In [ ]:
SubGush_table_cols = ['SUB_GUSH_ID', 'GUSH_NUM', 'GUSH_SUFFIX', 'STATUS', 'STATUS_DATE', 'HESDER_LAST_PARCEL']
SubGush_table = pd.read_csv(RawData_folder + 'Oracle\sub_gush.csv', usecols = SubGush_table_cols, encoding = 'cp1255')

<h1 style="text-align:right;font-size:100%">
נתוני שכבת גושים

In [ ]:
SubGush_layer_cols = ['SUB_GUSH_ID_1', 'GUSH_NUM', 'GUSH_SUFFIX', 'geometry']
SubGush_layer = gpd.read_file(RawData_folder + 'ArcStorm\SUB_GUSH_ALL.gdb', layer = 'SUB_GUSH_ALL', include_fields = SubGush_layer_cols)
SubGush_layer.rename(columns = {'SUB_GUSH_ID_1' : 'SUB_GUSH_ID'}, inplace = True)

<h1 style="text-align:right;font-size:100%">
שכבת מפתח גליונות גושים

In [ ]:
SheetKey_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'geometry']
SheetKey = gpd.read_file(RawData_folder + 'ArcStorm\SHEET_K.gdb', layer = 'SHEET_K', include_fields = SheetKey_cols)
SheetKey.rename(columns = {'geometry' : 'Sheet_geometry'}, inplace = True)

<h1 style="text-align:right;font-size:100%">
טבלת חלקות אחרונות בגושים מאורקל

In [ ]:
SubMMG_cols = ['SUB_GUSH_ID', 'LAST_PARCEL', 'MMG_LAST_PARCEL', 'COURT_LAST_PARCEL', 'FINAL_CONFIRMATION_DATE']
SubMMG = pd.read_csv(RawData_folder + 'Oracle\sub_gush_mmg.csv', usecols = SubMMG_cols, encoding = 'cp1255')

In [ ]:
del [RawData_folder, SubGush_layer_cols, SubGush_table_cols, SheetKey_cols, SubMMG_cols]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  1.b Cleaning data')

<h1 style="text-align:right;font-size:100%">
הסרת גושים מכל מקורות המידע שמספר הגוש שלהם הוא 0 או ריק

In [ ]:
SubGush_layer = SubGush_layer[(SubGush_layer['GUSH_NUM'] != 0) | (~SubGush_layer['GUSH_NUM'].isna())]
SubGush_table = SubGush_table[(SubGush_table['GUSH_NUM'] != 0) | (~SubGush_table['GUSH_NUM'].isna())]
SheetKey      = SheetKey     [(SheetKey['GUSH_NUM'] != 0)      | (~SheetKey['GUSH_NUM'].isna())]

<h1 style="text-align:right;font-size:100%">
מילוי ערכים ריקים של שדה תת-גוש בערך 0 לכל מקורות המידע

In [ ]:
SubGush_layer['GUSH_SUFFIX'] = SubGush_layer['GUSH_SUFFIX'].fillna(0)
SubGush_table['GUSH_SUFFIX'] = SubGush_table['GUSH_SUFFIX'].fillna(0)
SheetKey['GUSH_SUFFIX']      = SheetKey['GUSH_SUFFIX'].fillna(0)

<h1 style="text-align:right;font-size:100%">
הסרת סטאטוסים לא ידועים מטבלת גושים באורקל 

In [ ]:
SubGush_table = SubGush_table[~SubGush_table['STATUS'].isin([0, 99, None, np.nan])]

<h1 style="text-align:right;font-size:100%">
הגדרות שדות של תאריכים לסוג תאריך

In [ ]:
SubGush_table['STATUS_DATE']      = pd.to_datetime(SubGush_table['STATUS_DATE'], errors = 'coerce')
SubMMG['FINAL_CONFIRMATION_DATE'] = pd.to_datetime(SubMMG['FINAL_CONFIRMATION_DATE'], errors = 'coerce')

<h1 style="text-align:right;font-size:100%">
יצירת שדה מקשר של שם גוש מלא לכל מקורות המידע על מנת להוציא גושים משכבת מפתח גליונות שכבר נמצאים בשכבת גושים

In [ ]:
SubGush_table[['GUSH_NUM', 'GUSH_SUFFIX']] = SubGush_table[['GUSH_NUM', 'GUSH_SUFFIX']].astype(int)
SubGush_layer[['GUSH_NUM', 'GUSH_SUFFIX']] = SubGush_layer[['GUSH_NUM', 'GUSH_SUFFIX']].astype(int)
SheetKey[['GUSH_NUM', 'GUSH_SUFFIX']] = SheetKey[['GUSH_NUM', 'GUSH_SUFFIX']].astype(int)

SheetKey['BlockName'] = SheetKey['GUSH_NUM'].astype(str) + '-' + SheetKey['GUSH_SUFFIX'].astype(str)
SubGush_layer['BlockName'] = SubGush_layer['GUSH_NUM'].astype(str) + '-' + SubGush_layer['GUSH_SUFFIX'].astype(str)

SheetKey = SheetKey[~SheetKey['BlockName'].isin(SubGush_layer['BlockName'])]

SheetKey.drop(columns = 'BlockName', inplace = True)
SubGush_layer.drop(columns = 'BlockName', inplace = True)

In [ ]:
print('  1.c Data conversion')

<h1 style="text-align:right;font-size:100%">
מיזוג כל הנתונים
<h1 style="text-align:right;font-size:75%">
(ישנם מספרי מזהה של גושים בטבלת גושים של אורקל שלא קיים להם מידע במקורות אחרים, גושים אלו יושמטו באיחוד

In [ ]:
Blocks_all = SubGush_table.merge(right = SubGush_layer, on = ['SUB_GUSH_ID' , 'GUSH_NUM', 'GUSH_SUFFIX'], how = 'left')\
                          .merge(right = SheetKey, on = ['GUSH_NUM', 'GUSH_SUFFIX'], how = 'left')\
                          .merge(right = SubMMG, on = 'SUB_GUSH_ID', how = 'left')

del [SubGush_table, SubMMG, SheetKey]

<h1 style="text-align:right;font-size:100%">
מילוי ערכי ריקים של שדה גיאומטריה בעזרת גיאומטריה שמגיעה ממפתח גליונות
<h1 style="text-align:right;font-size:100%">
קיבוץ מרחבי לפי שדה של מספר מזהה לגוש בכדי למנוע יישויות גושים מפוצלים (מולטי-פארטס)

In [ ]:
Blocks_all['geometry'] = np.where(Blocks_all['geometry'].isna(),
                                  Blocks_all['Sheet_geometry'],
                                  Blocks_all['geometry'])

Blocks_all.drop(columns = ['Sheet_geometry'], inplace = True)

Blocks_all = gpd.GeoDataFrame(data = Blocks_all, geometry = 'geometry', crs = 2039)
Blocks_all = Blocks_all.dissolve(by = 'SUB_GUSH_ID', as_index = False, dropna = False)

<h1 style="text-align:right;font-size:100%">
STATUS הגדרת שדה סטאטוס חדש לגוש בעזרת מיפוי ערכי שדה

In [ ]:
Status_classifier = {
                     1  : 11,  #חדש רשום = נוצר בהסדר
                     3  : 11,  #לא רשום = נוצר בהסדר
                     4  : 21,  # לא מוסדר = לא מוסדר
                     5  : 41,  # מבוטל =  מוסדר מבוטל
                     6  : 11,  # מוסדר = נוצר בהסדר
                     7  : 22,  # בהסדר = בהסדר חלוקה
                     9  : 27,  # רישום ראשון = רישום ראשון
                     10 : 22,  # חלוקה = בהסדר חלוקה
                     11 : 23,  # מוקדמת = בהסדר מוקדמת
                     13 : 24,  # ארעית = בהסדר ארעית
                     15 : 25,  # סופית = בהסדר סופית
                     16 : 11,  # ירדני = נוצר  בהסדר
                     17 : 22,  # ירדני בהסדר = בהסדר חלוקה
                     18 : 28,  # רישום בשטח לא מוסדר = רישום בשטח לא מוסדר
                     51 : 41,  # חדש מבוטל = מוסדר מבוטל
                     56 : 42,  # הסדר מבוטל = בהסדר מבוטל
                     57 : 42,  # מתבטל הסדר = בהסדר מבוטל
                     58 : 45,  # גוש בשטח לא מוסדר - מבוטל = רישום בשטח לא מוסדר מבוטל
                     None : 26 # ללא ערך = הסדר בהקפאה
                    }

Blocks_all['BlockStatus'] = Blocks_all['STATUS'].map(Status_classifier)
Blocks_all['BlockStatus'] = Blocks_all['BlockStatus'].astype(int)

del Status_classifier

<h1 style="text-align:right;font-size:100%">
STATUS הגדרת שדה סוג מקרקעין  לגוש בעזרת מיפוי ערכי שדה

In [ ]:
registered_LandTypes   = [1, 3, 6, 16] # [חדש רשום, לא רשום, מוסדר, ירדני]

register_code   = 1
unregister_code = 2

LandType_cond   = [Blocks_all['STATUS'].isin(registered_LandTypes), ~Blocks_all['STATUS'].isin(registered_LandTypes)]
LandType_choise = [register_code, unregister_code]

Blocks_all['LandType'] = np.select(condlist   = LandType_cond,
                                   choicelist = LandType_choise,
                                   default    = unregister_code)

del [registered_LandTypes, register_code, unregister_code]

<h1 style="text-align:right;font-size:100%">
STATUS הגדרת שדה בינארי לגוש ירדני בעזרת מיפוי ערך 16 בשדה

In [ ]:
IsJordanian_code    = 1
IsNotJordanian_code = 0

Blocks_all['IsJordanian'] = np.where(Blocks_all['STATUS'] == 16, IsJordanian_code, IsNotJordanian_code)

del [IsJordanian_code, IsNotJordanian_code]

In [ ]:
Blocks_all.drop(columns = 'STATUS', inplace = True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  1.d Defining new Layer')

In [ ]:
Blocks_all.STATUS_DATE.dtype

In [ ]:
new_columns = np.dtype([
                        ("BlockUniqueID", int),
                        ('BlockNumber', int),
                        ("SubBlockNumber", int),
                        ("BlockStatus", int),
                        ("IsJordanian", int),
                        ("LandType", int),
                        ("SetteledDate", 'datetime64[D]'),
                        ("UpdateDate", 'datetime64[D]'),
                        ("LastRegisterdParcel", int),
                        ("LastParcel", int),
                        ("LastSetteledParcel", int),
                        ("LastCourtParcel", int)
                        ])


Blocks = pd.DataFrame(np.empty(0, dtype = new_columns))

<h1 style="text-align:right;font-size:100%">
איכלוס שדות לגושים מוסדרים ולא מוסדרים

In [ ]:
Blocks['BlockUniqueID'] = Blocks_all['SUB_GUSH_ID'].astype(int)


Blocks['BlockNumber'] = Blocks_all['GUSH_NUM'].astype(int)


Blocks['SubBlockNumber'] = Blocks_all['GUSH_SUFFIX'].fillna(0)
Blocks['SubBlockNumber'] = Blocks['SubBlockNumber'].astype(int)


Blocks['BlockStatus'] = Blocks_all['BlockStatus'].astype(int)


Blocks['IsJordanian'] = Blocks_all['IsJordanian'].astype(int)


Blocks['LandType'] = Blocks_all['LandType'].astype(int)


Blocks['SetteledDate'] = Blocks_all['STATUS_DATE'].astype('datetime64[D]')


Blocks['UpdateDate'] = Blocks_all['FINAL_CONFIRMATION_DATE'].astype('datetime64[D]')


Blocks['LastRegisterdParcel'] = Blocks_all['LAST_PARCEL'].astype('Int64')


Blocks['LastParcel'] = Blocks_all['MMG_LAST_PARCEL'].astype('Int64')


Blocks['LastSetteledParcel'] = Blocks_all['HESDER_LAST_PARCEL'].astype('Int64')


Blocks['LastCourtParcel'] = Blocks_all['COURT_LAST_PARCEL'].astype('Int64')


Blocks['geometry'] = Blocks_all['geometry']

<h1 style="text-align:right;font-size:100%">    
מיון הטבלה לפי מספר מזהה גוש

In [ ]:
Blocks.sort_values('BlockUniqueID', inplace = True)

del Blocks_all

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות לגושים מוסדרים/לא מוסדרים 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;">     
פיצול התוצאות לפי יישויות בעלי גיאומטריה וללא גיאומטריה    
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  1.e Exporting results to product folder')

In [ ]:
Blocks_geom = gpd.GeoDataFrame(Blocks[~Blocks['geometry'].isna()],
                               geometry = 'geometry',
                               crs = 2039)

Blocks_no_geom = Blocks[Blocks['geometry'].isna()]

In [ ]:
no_geom_csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\Blocks_no_geom.csv'
json_dir        = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\Blocks_geom.geojson'


Blocks_no_geom.drop(columns = 'geometry').to_csv(no_geom_csv_dir , index = False)
Blocks_geom.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\Blocks-Save as Feature Class.bat"]);